In [20]:
import tensorflow as tf
import cv2
from tqdm import tqdm
import os
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Input
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from IPython.display import display, Image
import matplotlib.pyplot as plt

In [2]:
DATASET_DIRECTORY = "C:/Users/cauld/Downloads/face-mask-detector/face-mask-detector/dataset"
CATEGORIES = ['without_mask', 'with_mask']
IMG_SIZE = 224

In [3]:
images = []
labels = []

aug_generate = ImageDataGenerator(
    rotation_range=20,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode='nearest',
)

def create_data():
    for label_number, category in enumerate(CATEGORIES):
        path = os.path.join(DATASET_DIRECTORY, category)
        for img in tqdm(os.listdir(path)):
            image = load_img(os.path.join(path, img), target_size=(IMG_SIZE, IMG_SIZE))
            image = img_to_array(image)
            image = preprocess_input(image)
            images.append(image)
            labels.append(label_number)

create_data()
images = np.array(images, dtype="float32")
labels = np.array(labels)

100%|███████████████████████████████████████████████████████████████████████████████| 690/690 [00:05<00:00, 122.52it/s]


In [4]:
images.shape

(1376, 224, 224, 3)

In [5]:
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
labels = tf.keras.utils.to_categorical(labels)

In [6]:
images_train, images_test, labels_train, labels_test = train_test_split(images, labels, train_size=0.80, test_size=0.20, random_state=42)

In [130]:
len(images_train)

1100

In [7]:
baseModel = MobileNetV2(weights="imagenet", include_top=False, input_tensor=Input(shape=(IMG_SIZE, IMG_SIZE, 3)))

headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(7, 7))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(128, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(2, activation="softmax")(headModel)

C:\Users\cauld\anaconda3\envs\tensorflow\lib\site-packages\keras_applications\mobilenet_v2.py:294: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  warnings.warn('`input_shape` is undefined or non-square, '


In [8]:
model = tf.keras.models.Model(inputs=baseModel.input, outputs=headModel)

for layer in baseModel.layers:
    layer.trainable = False

In [9]:
INIT_LR = 1e-4
EPOCHS = 20
BS = 32
opt = tf.keras.optimizers.Adam(learning_rate=INIT_LR, decay=INIT_LR/EPOCHS)

model.compile(loss='binary_crossentropy', optimizer=opt, metrics =['accuracy'])
model.fit(aug_generate.flow(images_train, labels_train, batch_size=BS),
            steps_per_epoch=len(images_train) // BS,
            validation_data=(images_test, labels_test),
            validation_steps=len(images_test) // BS,
            epochs = EPOCHS)

  ...
    to  
  ['...']
Train for 34 steps, validate on 276 samples
Epoch 1/20
34/34 [==============================] - 14s 412ms/step - loss: 0.7156 - accuracy: 0.6011 - val_loss: 0.3677 - val_accuracy: 0.8711
Epoch 2/20
34/34 [==============================] - 10s 298ms/step - loss: 0.4074 - accuracy: 0.8165 - val_loss: 0.2258 - val_accuracy: 0.9219
Epoch 3/20
34/34 [==============================] - 10s 300ms/step - loss: 0.3064 - accuracy: 0.8670 - val_loss: 0.1623 - val_accuracy: 0.9531
Epoch 4/20
34/34 [==============================] - 11s 311ms/step - loss: 0.2274 - accuracy: 0.9082 - val_loss: 0.1323 - val_accuracy: 0.9570
Epoch 5/20
34/34 [==============================] - 11s 314ms/step - loss: 0.1795 - accuracy: 0.9307 - val_loss: 0.1229 - val_accuracy: 0.9492
Epoch 6/20
34/34 [==============================] - 10s 308ms/step - loss: 0.1977 - accuracy: 0.9298 - val_loss: 0.1003 - val_accuracy: 0.9688
Epoch 7/20
34/34 [==============================] - 10s 308ms/step - loss

In [13]:
loss, accuracy = model.evaluate(images_test, labels_test)

276/276 [==============================] - 1s 2ms/sample - loss: 0.0551 - accuracy: 0.9783


In [43]:
MODEL_PATH = "C:/Users/cauld/Documents/python/mask_classifier_model.h5"
model.save(MODEL_PATH)

In [84]:
CONFIDENCE = 0.5
FACE_DETECTOR = "C:/Users/cauld/Downloads/face-mask-detector/face-mask-detector/face_detector"
MODEL_PATH = "C:/Users/cauld/Documents/python"
IMG_PATH = 'C:/Users/cauld/Desktop/temp.jpg'

In [44]:
prototxtPath = os.path.sep.join([FACE_DETECTOR, "deploy.prototxt"])
weightsPath = os.path.sep.join([FACE_DETECTOR, "res10_300x300_ssd_iter_140000.caffemodel"])
net = cv2.dnn.readNet(prototxtPath, weightsPath)

model = tf.keras.models.load_model(MODEL_PATH)

In [85]:
image = cv2.imread(IMG_PATH)
orig = image.copy()
(h, w) = image.shape[:2]

blob = cv2.dnn.blobFromImage(image, 1.0, (300, 300), (104.0, 177.0, 123.0))
net.setInput(blob)
detections = net.forward()

In [86]:
for i in range(0, detections.shape[2]):
    confidence = detections[0, 0, i, 2]
    if confidence > CONFIDENCE:
        box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
        (startX, startY, endX, endY) = box.astype("int")
        (startX, startY) = (max(0, startX), max(0, startY))
        (endX, endY) = (min(w - 1, endX), min(h - 1, endY))
        face = image[startY:endY, startX:endX]
        face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
        face = cv2.resize(face, (224, 224))
        face = img_to_array(face)
        face = preprocess_input(face)
        face = np.expand_dims(face, axis=0)
        (withoutMask, mask) = model.predict(face)[0]
        
        label = "Mask" if mask > withoutMask else "No Mask"
        color = (0, 255, 0) if label == "Mask" else (0, 0, 255)
        label = "{}: {:.2f}%".format(label, max(mask, withoutMask) * 100)
        cv2.putText(image, label, (startX, startY - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
        cv2.rectangle(image, (startX, startY), (endX, endY), color, 2)
        
cv2.imshow("Output", image)
cv2.waitKey(0)

-1